In [6]:
using Revise, Statistics, LaTeXStrings, DSP, StatsBase
using Formatting, MAT, FFTW, Unitful
import Unitful: m, s, Hz
using CairoMakie, Makie
using ColorSchemes
import ColorSchemes: nord, tab10, Dark2_8

printx(x,N) = sprintf1("%.$(N)f",x)


# fg = :black; gg = :grey; cscheme = tab10
fg = nord[5]; gg = nord[3]; cscheme = Dark2_8
bg = :transparent


set_theme!(Theme(
    fontsize = 20,
    palette = ( color = cscheme,),
    markerstrokewidth=0.1,
    Axis = (
        backgroundcolor = bg,
        xtickcolor = fg, ytickcolor = fg,
        xgridcolor = gg, ygridcolor = gg,
        xlabelcolor = fg, ylabelcolor = fg,
        xticklabelcolor = fg, yticklabelcolor = fg,
        topspinecolor = fg, bottomspinecolor = fg,
        leftspinecolor = fg, rightspinecolor = fg,
        titlecolor = fg,
    ),
    Legend = (
        bgcolor = bg,
        labelcolor = fg, titlecolor = fg,
        framevisible = false, margin=(0,0,0,0)
    )
))

In [2]:
matfile = matread("mimo_project.mat")

Dict{String, Any} with 12 entries:
  "N_rx"      => 32.0
  "c"         => 340.0
  "tx_pos"    => [-0.0085 0.0085]
  "B"         => 10000.0
  "cdma_data" => ComplexF64[0.0+0.0im 0.0+0.0im … 0.0+0.0im 0.0+0.0im; 0.0+0.0i…
  "T_p"       => 0.01
  "fs"        => 200000.0
  "rx_pos"    => [-0.527 -0.493 … 0.493 0.527]
  "N_tx"      => 2.0
  "fc"        => 10000.0
  "tdma_data" => [0.0+0.0im 0.0+0.0im … 0.0+0.0im 0.0+0.0im; 0.0+0.0im 0.0+0.0i…
  "N_t"       => 10000.0

In [9]:
Nᵣₓ = matfile["N_rx"] |> Int
Nₜₓ = matfile["N_tx"] |> Int
Nₜ  = matfile["N_t"]  |> Int

c   = matfile["c"] * m/s
B   = matfile["B"] * Hz
fs  = matfile["fs"] * Hz
fc  = matfile["fc"] * Hz
Tₚ  = matfile["T_p"] * s

tₓ_pos = matfile["tx_pos"]
rₓ_pos = matfile["rx_pos"] 

tdma_data = matfile["tdma_data"]
cdma_data = matfile["cdma_data"];

In [14]:
# Create waveforms
α = B/Tₚ
S_up   = (t -> @. exp(0im*2π*((fc - B/2)*t + α*t^2/2)))(0s:1/fs:Tₚ);
S_down = (t -> @. exp(1im*2π*((fc + B/2)*t - α*t^2/2)))(0s:1/fs:Tₚ);

In [1]:
match_filter(x, y) = xcorr(x, y)

lines(match_filter(tdma_data[:,16,1], S_up) .|> abs)

UndefVarError: UndefVarError: tdma_data not defined